In [13]:
import os
import pandas as pd
import tensorflow as tf
import lightgbm as lightgbm
import xgboost as xgb
import matplotlib.pyplot as plt
import numpy as np
import datetime
import time
import seaborn as sns
import time
from joblib import dump, load
from datetime import timedelta
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn import svm
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import RegressorChain
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
pd.options.display.max_rows = 10


completeTimer = time.time()
typeTimer = time.time()
estimatorTimer = time.time()

# output log file
outF = open("output.txt", "w")


# environment
isSuperComputer = False

cantonKeys = ['AG','AI','AR', 'BE', 'BL', 'BS', 'FR', 'GE', 'GL', 'GR', 'JU', 'LU', 'NE', 'NW', 'OW', 'SG', 'SH', 'SO', 'SZ', 'TG', 'TI', 'UR', 'VD', 'VS', 'ZG','ZH']

data = pd.read_csv("completedata.csv")

# last check if there are some NaN values in the data which we use
for col in data.columns:
    if data[col].isna().sum() != 0:
        print(col+" (#NaN/#NotNaN): (" + str(data[col].isna().sum())+"/"+str(data[col].notna().sum())+")")


numberOfInputWeeks = 3
numberOfOutputWeeks = 1
#display([col for col in data.columns])


outputCategories = [
                  'case_inz_entries_7dayAverage',
                  'hosp_inz_entries_7dayAverage',
                  'death_inz_entries_7dayAverage',
                  'testPositvity',
                  'testPositvity_7dayAverageBoth',    
                  'transit_stations_percent_change_from_baseline_7dayAverage',
                  'workplaces_percent_change_from_baseline_7dayAverage',    
                   ]
numberOfOutputs = len(outputCategories)
numberOfPreComputedOutputWeeks = 4

split = numberOfOutputs * numberOfPreComputedOutputWeeks + 2
train_features = data[data['category']=='train'].iloc[:,0:-split].drop(['lastInputDay'], axis=1)
train_labels = data[data['category']=='train'].iloc[:,-split:-2]
validation1_features = data[data['category']=='validation 1'].iloc[:,0:-split].drop(['lastInputDay'], axis=1)
validation1_labels = data[data['category']=='validation 1'].iloc[:,-split:-2]
validation2_features = data[data['category']=='validation 2'].iloc[:,0:-split].drop(['lastInputDay'], axis=1)
validation2_labels = data[data['category']=='validation 2'].iloc[:,-split:-2]

'''
cId = 'AI'
train_features = data[(data['category']=='train')].iloc[:,0:-26]
train_labels = data[(data['category']=='train')].iloc[:,-26:-2]
validation1_features = data[(data['category']=='validation 1') & (data['cantonId']==cId)].iloc[:,0:-26]
validation1_labels = data[(data['category']=='validation 1') & (data['cantonId']==cId)].iloc[:,-26:-2]
validation2_features = data[(data['category']=='validation 2') & (data['cantonId']==cId)].iloc[:,0:-26]
validation2_labels = data[(data['category']=='validation 2') & (data['cantonId']==cId)].iloc[:,-26:-2]
'''


#pip = Pipeline([('std_scaler', StandardScaler())])
pip = Pipeline([('minmax_scaler', MinMaxScaler())])

X_train = pip.fit_transform(train_features[train_features.columns].values)
X_valid1 = pip.transform(validation1_features[train_features.columns].values)
X_valid2 = pip.transform(validation2_features[train_features.columns].values)


In [14]:
modelId = 0
# (str(type(estimator)) == "<class 'tensorflow.python.keras.engine.functional.Functional'>")

results = pd.DataFrame(columns=['model',
                                'model type',
                                'target',
                                'week',
                                'model rsme 1', 
                                'model rsme 2'])

# --------------------------------------------------------------------------------------------------------
typeTimer = time.time()
display("============== fitting normal sklearn estimators ==============") 
outF.write("============== fitting normal sklearn estimators ============== \n")

# fitting normal sklearn estimators

estimators = [
    linear_model.Ridge(alpha=1),
    linear_model.Lasso(alpha=1),
]


if isSuperComputer:
    # all all estimators which we want to search through
    estimators = []
    
    # add LGBMs
    for regalpha in [0.001,0.01,0.1,0,1,10,100]:
        for reglambda in [0.001,0.01,0.1,0,1,10,100]:
            estimators.append(lightgbm.LGBMRegressor(n_estimators=10000, reg_alpha=regalpha, reg_lambda=reglambda))
       
    # add XGBoosts
    for regalpha in [0.001,0.01,0.1,0,1,10,100]:
        for reglambda in [0.001,0.01,0.1,0,1,10,100]: 
            estimators.append(xgb.XGBRegressor(n_estimators=1000, reg_alpha=regalpha, reg_lambda=reglambda))
    
    # add random forrest
    for nrOfEstimators in [100,200,300,500,1000]:
        estimators.append(RandomForestRegressor(n_estimators=nrOfEstimators))

    # add MLPs
    for alpha in [0.001,0.01,0.1,1,10,10]:
        for layer in [
            (50),
            (100),
            (20,20),
            (120,10),
            (100,10),
            (100,100),
            (120,100),
            (200,100,10),
            (200,100,100),
            (100,100,100),
            (10,10,10,10,5),
            (20,10,10,10,10),
        ]:
            estimators.append(MLPRegressor(hidden_layer_sizes=layer, alpha=alpha, max_iter=1000))
    
    # add SVRs
    for c in [0.001,0.01,0.1,1,10,100]:
        estimators.append(svm.SVR(C=c)) 
    
    
    # add linear models
    estimators.append(linear_model.Ridge(alpha=0))
    estimators.append(linear_model.Lasso(alpha=0)) 
    for alpha in [0.001,0.01,0.1,1,10,100]:
        estimators.append(linear_model.Ridge(alpha=alpha))
        estimators.append(linear_model.Lasso(alpha=alpha)) 
        for lr in ['adaptive','optimal','invscaling']:
            for p in ['l1','l2','elasticnet']:
                estimators.append(linear_model.SGDRegressor(learning_rate = lr,penalty=p, alpha=alpha,max_iter=10000, tol=0.00001, n_iter_no_change=10))
    
display("number of estimators to fit: " + str(len(estimators)*len(outputCategories)))
outF.write("number of estimators to fit: " + str(len(estimators)*len(outputCategories))+" \n")
display("number of outputs: " + str(len(outputCategories)))


for estimator in estimators:
    display(estimator)
    estimatorTimer = time.time()
    outF.write(str(estimator)+" \n")
    for category in outputCategories:
        display("-- " + category)
        outF.write("-- " + category+" \n")
        
        trainOutput = train_labels[["output_"+category+"_"+str(i) for i in range(0,numberOfOutputWeeks)]] 
        reg = estimator.fit(X_train, trainOutput.values)           
        # save the fitted regressor
        dump(reg, 'models/'+str(modelId)+'.joblib') 
        reg = load('models/'+str(modelId)+'.joblib') 
        modelId = modelId + 1
        
        predictions_valid1 = pd.DataFrame(reg.predict(X_valid1), index=validation1_labels.index, columns=["pred_week_" + str(i)+"_"+category for i in range(0,numberOfOutputWeeks)]) 
        predictions_valid2 = pd.DataFrame(reg.predict(X_valid2), index=validation2_labels.index, columns=["pred_week_" + str(i)+"_"+category for i in range(0,numberOfOutputWeeks)])
        
        valid1Output = validation1_labels[["output_"+category+"_"+str(i) for i in range(0,numberOfOutputWeeks)]]
        valid2Output = validation2_labels[["output_"+category+"_"+str(i) for i in range(0,numberOfOutputWeeks)]]
        
        for predictionWeek in range(0,numberOfOutputWeeks):
            rmse1 = np.sqrt(mean_squared_error(predictions_valid1["pred_week_" + str(predictionWeek)+"_"+category], valid1Output.iloc[:,predictionWeek]))
            rmse2 = np.sqrt(mean_squared_error(predictions_valid2["pred_week_" + str(predictionWeek)+"_"+category], valid2Output.iloc[:,predictionWeek]))
            results = results.append({'model':reg,
                                      'model type': "sklearn_single_output",
                                      'target':category,
                                      'week':predictionWeek, 
                                      'model rsme 1':rmse1, 
                                      'model rsme 2':rmse2
                                     }, ignore_index=True)
            
    outF.write(str(estimator)+" (%s) \n" % (time.time() - estimatorTimer))
            
outF.write("============== finished fitting normal sklearn estimators ============== (%s) \n" % (time.time() - typeTimer))           
# ------------------------------------------------------------------------------------------------                       
typeTimer = time.time()
display("============== fitting single output keras estimators ==============")
outF.write("============== fitting single output keras estimators ============== \n")
start = time.time()
# fitting single output keras estimators

numberOfOutputsForMultiOutput = numberOfOutputWeeks * len(outputCategories)
epochs = 100
batch_size = 32

def getNormalKerasModel(name, learningrate, nrOfNeurons, isMultiOutput):
    input_ = keras.layers.Input(shape=X_train.shape[1:])
    hidden1 = keras.layers.Dense(nrOfNeurons, activation="relu")(input_)
    output = keras.layers.Dense(numberOfOutputsForMultiOutput)(hidden1) if isMultiOutput else keras.layers.Dense(1)(hidden1)
    model = keras.Model(name=name, inputs=[input_],outputs=[output])
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
    return model


def getDropoutKerasModel(name, learningrate, nrOfNeurons, isMultiOutput):
    input_ = keras.layers.Input(shape=X_train.shape[1:])
    hidden1 = keras.layers.Dense(nrOfNeurons, activation="relu")(input_)
    dropout1 = keras.layers.Dropout(rate=0.1)(hidden1)
    output = keras.layers.Dense(numberOfOutputsForMultiOutput)(hidden1) if isMultiOutput else keras.layers.Dense(1)(hidden1)
    model = keras.Model(name=name, inputs=[input_],outputs=[output])
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
    return model

def getRegularizedKerasModel(name, learningrate, nrOfNeurons, alpha, isMultiOutput):
    input_ = keras.layers.Input(shape=X_train.shape[1:])
    hidden1 = keras.layers.Dense(nrOfNeurons, activation="relu", kernel_regularizer=keras.regularizers.l2(alpha))(input_)
    output = keras.layers.Dense(numberOfOutputsForMultiOutput)(hidden1) if isMultiOutput else keras.layers.Dense(1)(hidden1)
    model = keras.Model(name=name, inputs=[input_],outputs=[output])
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
    return model

def getTwoWayKerasModel(name, learningrate, nrOfNeurons, isMultiOutput):
    input_ = keras.layers.Input(shape=X_train.shape[1:])
    hidden1 = keras.layers.Dense(nrOfNeurons, activation="relu")(input_)
    concat = keras.layers.Concatenate()([input_,hidden1])
    output = keras.layers.Dense(numberOfOutputsForMultiOutput)(concat) if isMultiOutput else keras.layers.Dense(1)(concat)
    model = keras.Model(name=name, inputs=[input_],outputs=[output])
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
    return model

def getComplexKerasModel(name, learningrate, nrOfNeurons1, nrOfNeurons2, isMultiOutput):
    input_ = keras.layers.Input(shape=X_train.shape[1:])
    hidden1 = keras.layers.Dense(nrOfNeurons1, activation="relu")(input_)
    hidden2 = keras.layers.Dense(nrOfNeurons2, activation="relu")(hidden1)
    output = keras.layers.Dense(numberOfOutputsForMultiOutput)(hidden1) if isMultiOutput else keras.layers.Dense(1)(hidden2)
    model = keras.Model(name=name, inputs=[input_],outputs=[output])
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
    return model



estimators = []

estimators.append(getNormalKerasModel("simple", 0.0001, 10, False))

if isSuperComputer:
    # all all estimators which we want to search through
    estimators = []
    
    for learningrate in [0.001,0.0001,0.00001]:
        estimators.append(getNormalKerasModel("normal_learningrate_"+str(learningrate), learningrate, 100, False))
        estimators.append(getDropoutKerasModel("dropout_learningrate_"+str(learningrate), learningrate, 100, False))
        estimators.append(getTwoWayKerasModel("twoway_learningrate_"+str(learningrate), learningrate, 100, False))
        estimators.append(getComplexKerasModel("complex_learningrate_"+str(learningrate), learningrate, 100, 10, False))
        
        for alpha in [0.01, 0.001, 0.0001]:
            estimators.append(getRegularizedKerasModel("regularized_learningrate_"+str(learningrate)+"_alpha_"+str(alpha), learningrate, 100, alpha, False))     
    
    
display("number of estimators to fit: " + str(len(estimators)*len(outputCategories)))
outF.write("number of estimators to fit: " + str(len(estimators)*len(outputCategories))+" \n")
start = time.time()
display("number of outputs: " + str(len(outputCategories)))
    
for estimator in estimators:
    estimatorTimer = time.time()
    display(estimator.name)
    outF.write(str(estimator.name)+" \n")
    for category in outputCategories:
        
        display("-- " + category)
        outF.write("-- " + category+" \n")
        
        y_train = train_labels[["output_"+category+"_"+str(i) for i in range(0,numberOfOutputWeeks)]].values 
        y_valid1 = validation1_labels[["output_"+category+"_"+str(i) for i in range(0,numberOfOutputWeeks)]].values 
        estimator.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=0, validation_data=(X_valid1,y_valid1), callbacks=[keras.callbacks.EarlyStopping(patience=10)])
        reg = estimator
      
        # save the fitted regressor
        reg.save('models/'+str(modelId)+'.h5')
        reg = keras.models.load_model('models/'+str(modelId)+'.h5')
        modelId = modelId + 1
        
        predictions_valid1 = pd.DataFrame(reg.predict(X_valid1), index=validation1_labels.index, columns=["pred_week_" + str(i)+"_"+category for i in range(0,numberOfOutputWeeks)]) 
        predictions_valid2 = pd.DataFrame(reg.predict(X_valid2), index=validation2_labels.index, columns=["pred_week_" + str(i)+"_"+category for i in range(0,numberOfOutputWeeks)])
        
        valid1Output = validation1_labels[["output_"+category+"_"+str(i) for i in range(0,numberOfOutputWeeks)]]
        valid2Output = validation2_labels[["output_"+category+"_"+str(i) for i in range(0,numberOfOutputWeeks)]]
        
        for predictionWeek in range(0,numberOfOutputWeeks):
            rmse1 = np.sqrt(mean_squared_error(predictions_valid1["pred_week_" + str(predictionWeek)+"_"+category], valid1Output.iloc[:,predictionWeek]))
            rmse2 = np.sqrt(mean_squared_error(predictions_valid2["pred_week_" + str(predictionWeek)+"_"+category], valid2Output.iloc[:,predictionWeek]))
            results = results.append({'model':reg.name,
                                      'model type': "keras_single_output",
                                      'target':category,
                                      'week':predictionWeek, 
                                      'model rsme 1':rmse1, 
                                      'model rsme 2':rmse2
                                     }, ignore_index=True)
    outF.write(str(estimator.name)+" (%s) \n" % (time.time() - estimatorTimer))
        
outF.write("============== finished fitting single output keras estimators ============== (%s) \n" % (time.time() - typeTimer))   
# ----------------------------------------------------------------------------------------------------
typeTimer = time.time()
display("============== fitting multi output keras estimators ==============")
outF.write("============== fitting multi output keras estimators ============== \n")

estimators = []

estimators.append(getNormalKerasModel("simple", 0.0001, 10, True))

if isSuperComputer:
    # all all estimators which we want to search through
    estimators = []
    
    for learningrate in [0.001,0.0001,0.00001]:
        estimators.append(getNormalKerasModel("normal_learningrate_"+str(learningrate), learningrate, 100, True))
        estimators.append(getDropoutKerasModel("dropout_learningrate_"+str(learningrate), learningrate, 100, True))
        estimators.append(getTwoWayKerasModel("twoway_learningrate_"+str(learningrate), learningrate, 100, True))
        estimators.append(getComplexKerasModel("complex_learningrate_"+str(learningrate), learningrate, 100, 10, True))
        
        for alpha in [0.01, 0.001, 0.0001]:
            estimators.append(getRegularizedKerasModel("regularized_learningrate_"+str(learningrate)+"_alpha_"+str(alpha), learningrate, 100, alpha, True))     
            
display("number of estimators to fit: " + str(len(estimators)))
outF.write("number of estimators to fit: " + str(len(estimators))+" \n")
start = time.time()
display("number of outputs: " + str(len(outputCategories)))

for estimator in estimators:
    estimatorTimer = time.time()
    display(estimator.name)
    outF.write(str(estimator.name)+" \n")
    display("-- all categories")
    outF.write("-- all categories \n")
    
    
    y_train = train_labels[["output_"+category+"_"+str(i) for category in outputCategories for i in range(0,numberOfOutputWeeks)]].values
    y_valid1 = validation1_labels[["output_"+category+"_"+str(i) for category in outputCategories for i in range(0,numberOfOutputWeeks)]].values 
    estimator.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=0, validation_data=(X_valid1,y_valid1), callbacks=[keras.callbacks.EarlyStopping(patience=10)])
    reg = estimator
    
    # save the fitted regressor
    reg.save('models/'+str(modelId)+'.h5')
    reg = keras.models.load_model('models/'+str(modelId)+'.h5')
    modelId = modelId + 1
    
    predictions_valid1 = pd.DataFrame(reg.predict(X_valid1), index=validation1_labels.index, columns=["pred_week_" + str(i)+"_"+category for category in outputCategories for i in range(0,numberOfOutputWeeks)]) 
    predictions_valid2 = pd.DataFrame(reg.predict(X_valid2), index=validation2_labels.index, columns=["pred_week_" + str(i)+"_"+category for category in outputCategories for i in range(0,numberOfOutputWeeks)])
    
    valid1Output = validation1_labels[["output_"+category+"_"+str(i) for category in outputCategories for i in range(0,numberOfOutputWeeks)]]
    valid2Output = validation2_labels[["output_"+category+"_"+str(i) for category in outputCategories for i in range(0,numberOfOutputWeeks)]]

    for predictionWeek in range(0,numberOfOutputWeeks):
        for category in outputCategories:
            rmse1 = np.sqrt(mean_squared_error(predictions_valid1["pred_week_" + str(predictionWeek)+"_"+category], valid1Output["output_"+category+"_"+str(predictionWeek)]))
            rmse2 = np.sqrt(mean_squared_error(predictions_valid2["pred_week_" + str(predictionWeek)+"_"+category], valid2Output["output_"+category+"_"+str(predictionWeek)]))
            
            results = results.append({'model':reg.name,
                                      'model type': "keras_multi_output",
                                      'target':category,
                                      'week':predictionWeek, 
                                      'model rsme 1':rmse1, 
                                      'model rsme 2':rmse2}, ignore_index=True)
    outF.write(str(estimator.name)+" (%s) \n" % (time.time() - estimatorTimer))
    
outF.write("============== finished fitting multi output keras estimators ============== (%s) \n" % (time.time() - typeTimer))
# ----------------------------------------------------------------------------------------------------
typeTimer = time.time()
display("============== fitting multioutput sklearn estimators ==============")
outF.write("============== fitting multioutput sklearn estimators ============== (%s) \n" % (time.time() - start))
# fitting multioutput sklearn estimators
estimators = [
    linear_model.MultiTaskLasso(),
]
if isSuperComputer:
    # all all estimators which we want to search through
    
    kernels = ['linear','poly','polynomial','rbf','laplacian','sigmoid','cosine']
    
    estimators = []
    
    for k in kernels:
        estimators.append(KernelRidge(kernel=k))
    
    for nrOfEstimators in [10,100,1000]:
        for maxdepth in [None,10,100, 1000]:
            for mss in [2,3,4]:
                for msl in [1,10,100]:
                    estimators.append(RandomForestRegressor(n_estimators=nrOfEstimators, 
                                                            max_depth=maxdepth, 
                                                            min_samples_split=mss, 
                                                            min_samples_leaf=msl))
    for alpha in [0.001,0.01,0.1,1,10,10]:
        for layer in [
            (50),
            (100),
            (20,20),
            (120,10),
            (100,10),
            (100,100),
            (120,100),
            (200,100,10),
            (200,100,100),
            (100,100,100),
            (10,10,10,10,5),
            (20,10,10,10,10),
        ]:
            estimators.append(MLPRegressor(hidden_layer_sizes=layer, alpha=alpha, max_iter=1000))
    
    
    for alpha in [0.001,0.01,0.1,0,1,10,100]:
        estimators.append(linear_model.Lasso(alpha=alpha))
        estimators.append(linear_model.Ridge(alpha=alpha))
        estimators.append(linear_model.MultiTaskLasso(alpha=alpha))
    
display("number of estimators to fit: " + str(len(estimators)))
outF.write("number of estimators to fit: " + str(len(estimators))+" \n")
display("number of outputs: " + str(len(outputCategories)))


for estimator in estimators:
    estimatorTimer = time.time()
    display(estimator)
    outF.write(str(estimator) + " \n")
    display("-- all categories")
    outF.write("-- all categories \n")
    
    
    trainOutput = train_labels[["output_"+category+"_"+str(i) for category in outputCategories for i in range(0,numberOfOutputWeeks)]]
    reg = estimator.fit(X_train, trainOutput.values)
    # save the fitted regressor
    dump(reg, 'models/'+str(modelId)+'.joblib') 
    reg = load('models/'+str(modelId)+'.joblib') 
    modelId = modelId + 1
    
    predictions_valid1 = pd.DataFrame(reg.predict(X_valid1), index=validation1_labels.index, columns=["pred_week_" + str(i)+"_"+category for category in outputCategories for i in range(0,numberOfOutputWeeks)]) 
    predictions_valid2 = pd.DataFrame(reg.predict(X_valid2), index=validation2_labels.index, columns=["pred_week_" + str(i)+"_"+category for category in outputCategories for i in range(0,numberOfOutputWeeks)])
    
    valid1Output = validation1_labels[["output_"+category+"_"+str(i) for category in outputCategories for i in range(0,numberOfOutputWeeks)]]
    valid2Output = validation2_labels[["output_"+category+"_"+str(i) for category in outputCategories for i in range(0,numberOfOutputWeeks)]]

    
    for predictionWeek in range(0,numberOfOutputWeeks):
        for category in outputCategories:
            rmse1 = np.sqrt(mean_squared_error(predictions_valid1["pred_week_" + str(predictionWeek)+"_"+category], valid1Output["output_"+category+"_"+str(predictionWeek)]))
            rmse2 = np.sqrt(mean_squared_error(predictions_valid2["pred_week_" + str(predictionWeek)+"_"+category], valid2Output["output_"+category+"_"+str(predictionWeek)]))
            
            results = results.append({'model':reg,
                                      'model type': "sklearn_multi_output",
                                      'target':category,
                                      'week':predictionWeek, 
                                      'model rsme 1':rmse1, 
                                      'model rsme 2':rmse2}, ignore_index=True)
        
    outF.write(str(estimator)+" (%s) \n" % (time.time() - estimatorTimer))
        
outF.write("============== finished fitting multioutput sklearn estimators ============== (%s) \n" % (time.time() - typeTimer))
# -----------------------------------------------------------------------------------------------------

display("============== attach the baseline values ==============")
outF.write("============== attach the baseline values ============== \n")
# attach the baseline values
for predictionWeek in range(0,numberOfOutputWeeks):
    for category in outputCategories:
        # last value of last input week
        last_value_baseline_rmse1 = np.sqrt(mean_squared_error(validation1_features[category+"_last_"+str(numberOfInputWeeks-1)],
                                                               validation1_labels["output_"+category+"_"+str(predictionWeek)]))
        last_value_baseline_rmse2 = np.sqrt(mean_squared_error(validation2_features[category+"_last_"+str(numberOfInputWeeks-1)],
                                                               validation2_labels["output_"+category+"_"+str(predictionWeek)]))
        
        estimatorRows = results[(results['week']==predictionWeek) & (results['target']==category)]

        results.loc[estimatorRows.index, 'last value baseline rsme 1'] = last_value_baseline_rmse1
        results.loc[estimatorRows.index, 'last value baseline rsme 2'] = last_value_baseline_rmse2




results.index.rename("estimator")
results.to_csv('results.csv')


results['diff to last value baseline 1'] = (results['last value baseline rsme 1']-results['model rsme 1'])/results['last value baseline rsme 1']
results = results.sort_values(['diff to last value baseline 1'], ascending=False)



for category in outputCategories: 
    display(results[results['target']==category])   

outF.write("============== finished ============== %s \n" % (time.time() - completeTimer))
outF.close()


'============== fitting normal sklearn estimators =============='

'number of estimators to fit: 14'

'number of outputs: 7'

Ridge(alpha=1)

'-- case_inz_entries_7dayAverage'

...

,model,model type,target,week,model rsme 1,model rsme 2,last value baseline rsme 1,last value baseline rsme 2,diff to last value baseline 1
2,Ridge(alpha=1),sklearn_single_output,death_inz_entries_7dayAverage,0,0.138689,0.108794,0.1571,0.287626,0.117193
16,simple,keras_single_output,death_inz_entries_7dayAverage,0,0.230875,0.166873,0.1571,0.287626,-0.469610
9,Lasso(alpha=1),sklearn_single_output,death_inz_entries_7dayAverage,0,0.252222,0.433420,0.1571,0.287626,-0.605489
30,MultiTaskLasso(),sklearn_multi_output,death_inz_entries_7dayAverage,0,0.332808,0.320392,0.1571,0.287626,-1.118450
23,simple,keras_multi_output,death_inz_entries_7dayAverage,0,2.202390,0.867477,0.1571,0.287626,-13.019074


,model,model type,target,week,model rsme 1,model rsme 2,last value baseline rsme 1,last value baseline rsme 2,diff to last value baseline 1
31,MultiTaskLasso(),sklearn_multi_output,testPositvity,0,0.043392,0.062943,0.032502,0.06991,-0.335065
3,Ridge(alpha=1),sklearn_single_output,testPositvity,0,0.043741,0.048593,0.032502,0.06991,-0.345821
10,Lasso(alpha=1),sklearn_single_output,testPositvity,0,0.046700,0.095748,0.032502,0.06991,-0.436867
17,simple,keras_single_output,testPositvity,0,0.061316,0.060430,0.032502,0.06991,-0.886550
24,simple,keras_multi_output,testPositvity,0,0.501697,0.210365,0.032502,0.06991,-14.436111


,model,model type,target,week,model rsme 1,model rsme 2,last value baseline rsme 1,last value baseline rsme 2,diff to last value baseline 1
4,Ridge(alpha=1),sklearn_single_output,testPositvity_7dayAverageBoth,0,0.023524,0.017003,0.009519,0.035338,-1.471287
11,Lasso(alpha=1),sklearn_single_output,testPositvity_7dayAverageBoth,0,0.040244,0.084519,0.009519,0.035338,-3.227759
32,MultiTaskLasso(),sklearn_multi_output,testPositvity_7dayAverageBoth,0,0.042750,0.037626,0.009519,0.035338,-3.491010
18,simple,keras_single_output,testPositvity_7dayAverageBoth,0,0.062386,0.029435,0.009519,0.035338,-5.553800
25,simple,keras_multi_output,testPositvity_7dayAverageBoth,0,0.116856,0.159228,0.009519,0.035338,-11.276082


,model,model type,target,week,model rsme 1,model rsme 2,last value baseline rsme 1,last value baseline rsme 2,diff to last value baseline 1
19,simple,keras_single_output,transit_stations_percent_change_from_baseline_...,0,6.585808,7.548023,5.260542,8.727526,-0.251926
26,simple,keras_multi_output,transit_stations_percent_change_from_baseline_...,0,7.952158,11.091514,5.260542,8.727526,-0.511661
33,MultiTaskLasso(),sklearn_multi_output,transit_stations_percent_change_from_baseline_...,0,10.034352,12.171860,5.260542,8.727526,-0.907475
12,Lasso(alpha=1),sklearn_single_output,transit_stations_percent_change_from_baseline_...,0,10.732219,12.829043,5.260542,8.727526,-1.040136
5,Ridge(alpha=1),sklearn_single_output,transit_stations_percent_change_from_baseline_...,0,10.901188,5.408643,5.260542,8.727526,-1.072256


,model,model type,target,week,model rsme 1,model rsme 2,last value baseline rsme 1,last value baseline rsme 2,diff to last value baseline 1
20,simple,keras_single_output,workplaces_percent_change_from_baseline_7dayAv...,0,5.749043,6.324582,4.53423,9.822916,-0.267920
27,simple,keras_multi_output,workplaces_percent_change_from_baseline_7dayAv...,0,6.943506,8.549471,4.53423,9.822916,-0.531353
13,Lasso(alpha=1),sklearn_single_output,workplaces_percent_change_from_baseline_7dayAv...,0,8.681198,9.208663,4.53423,9.822916,-0.914592
34,MultiTaskLasso(),sklearn_multi_output,workplaces_percent_change_from_baseline_7dayAv...,0,9.498738,8.335067,4.53423,9.822916,-1.094896
6,Ridge(alpha=1),sklearn_single_output,workplaces_percent_change_from_baseline_7dayAv...,0,12.741614,3.747551,4.53423,9.822916,-1.810094


In [ ]:
# inspect a regressor
'''
# get exact parameters of choice
identifier = 6



week = 0
display(regressors[identifier][0].get_params())

# if the regressor has a features importance attribute
df = pd.DataFrame(regressors[identifier][0].estimators_[week].feature_importances_.transpose(), 
                  index=train_features[inputDict[regressors[identifier][1]]].columns, 
                  columns=['importance']).sort_values(['importance'], 
                  ascending=False)
df.plot(kind='barh', y='importance', figsize=(10,200))
plt.show()


# if the regressor has coefficients attribute 
df = pd.DataFrame(regressors[identifier][0].estimators_[week].coef_.transpose(), 
                  index=train_features[inputDict[regressors[identifier][1]]].columns, 
                  columns=['coefficients']).sort_values(['coefficients'], 
                  ascending=False)
df.plot(kind='barh', y='coefficients', figsize=(10,200))
plt.show()


'''


In [ ]:
'''
for cantonId in cantonKeys:
    category = regressors[identifier][1]
    dfForPlot = regressors[identifier][2] 

    d = dfForPlot[dfForPlot['cantonId']==cantonId]
    d = d.drop(['cantonId'], axis=1).rolling(window=7).mean()
    d.plot(kind='line',figsize=(20,10), title=cantonId)
    plt.show()

    d = d[d[category+"_"+str(numberOfInputWeeks-1)].notna()]


    sns.set_theme(style="ticks", palette="pastel",rc={"figure.figsize":(20, 10)})
    sns.boxplot(palette=["m", "g", 'r'], data=d).set(title=cantonId)
    plt.show()
'''